<a href="https://colab.research.google.com/github/lcanjar/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [63]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [64]:
# Determine the shape of the dataframe
attrition_df.shape

(1470, 27)

In [65]:
# Review the features and data types
attrition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   Department                1470 non-null   object
 4   DistanceFromHome          1470 non-null   int64 
 5   Education                 1470 non-null   int64 
 6   EducationField            1470 non-null   object
 7   EnvironmentSatisfaction   1470 non-null   int64 
 8   HourlyRate                1470 non-null   int64 
 9   JobInvolvement            1470 non-null   int64 
 10  JobLevel                  1470 non-null   int64 
 11  JobRole                   1470 non-null   object
 12  JobSatisfaction           1470 non-null   int64 
 13  MaritalStatus             1470 non-null   object
 14  NumCompaniesWorked      

In [66]:
# What features are of type object
object_features = attrition_df.select_dtypes(include=['object'])
print(object_features.columns)

Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField',
       'JobRole', 'MaritalStatus', 'OverTime'],
      dtype='object')


In [67]:
# Check for null values
null_proportion = attrition_df.isna().sum() / len(attrition_df)
print(null_proportion)

Age                         0.0
Attrition                   0.0
BusinessTravel              0.0
Department                  0.0
DistanceFromHome            0.0
Education                   0.0
EducationField              0.0
EnvironmentSatisfaction     0.0
HourlyRate                  0.0
JobInvolvement              0.0
JobLevel                    0.0
JobRole                     0.0
JobSatisfaction             0.0
MaritalStatus               0.0
NumCompaniesWorked          0.0
OverTime                    0.0
PercentSalaryHike           0.0
PerformanceRating           0.0
RelationshipSatisfaction    0.0
StockOptionLevel            0.0
TotalWorkingYears           0.0
TrainingTimesLastYear       0.0
WorkLifeBalance             0.0
YearsAtCompany              0.0
YearsInCurrentRole          0.0
YearsSinceLastPromotion     0.0
YearsWithCurrManager        0.0
dtype: float64


In [68]:
# Determine the number of unique values in each column.
attrition_df.nunique().sort_values(ascending=True)

,0
Attrition,2
PerformanceRating,2
OverTime,2
MaritalStatus,3
BusinessTravel,3
Department,3
JobSatisfaction,4
RelationshipSatisfaction,4
JobInvolvement,4
EnvironmentSatisfaction,4


In [69]:
# Get the column names with exactly 2 unique values
features_with_2_unique_values = attrition_df.columns[attrition_df.nunique() == 2]
print(features_with_2_unique_values.tolist())

['Attrition', 'OverTime', 'PerformanceRating']


# Imbalanced data

In [70]:
# Count the numbmer of unique values in target Attrition
unique_counts = attrition_df['Attrition'].value_counts()
print(unique_counts)

Attrition
No     1233
Yes     237
Name: count, dtype: int64


In [71]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', "Department"]]


In [72]:
# Create a list of at least 10 column names to use as X data
attrition_X_features = ['EnvironmentSatisfaction', 'HourlyRate', 'JobSatisfaction', 'PerformanceRating', 'WorkLifeBalance', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'DistanceFromHome','JobInvolvement', 'RelationshipSatisfaction']


# Create X_df using your selected columns
X_df = attrition_df[attrition_X_features]

# Show the data types for X_df
X_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   EnvironmentSatisfaction   1470 non-null   int64
 1   HourlyRate                1470 non-null   int64
 2   JobSatisfaction           1470 non-null   int64
 3   PerformanceRating         1470 non-null   int64
 4   WorkLifeBalance           1470 non-null   int64
 5   YearsInCurrentRole        1470 non-null   int64
 6   YearsSinceLastPromotion   1470 non-null   int64
 7   DistanceFromHome          1470 non-null   int64
 8   JobInvolvement            1470 non-null   int64
 9   RelationshipSatisfaction  1470 non-null   int64
dtypes: int64(10)
memory usage: 115.0 KB


In [73]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [74]:
# Convert your X data to numeric data types however you see fit

# "If all features are already numeric (int64 or float64), you can proceed with your machine learning workflow without additional encoding" as verified in Xpert Learning Assistant

In [75]:
# Create a OneHotEncoder for the Department column
ohe = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
ohe.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
# Transform the training data
department_train_encoded = ohe.fit_transform(y_train[['Department']])
# Transform the testing data
department_test_encoded = ohe.transform(y_test[['Department']])

# Create new variables to hold the encoded results
# Convert the encoded arrays to DataFrames for easier handling
department_train_encoded_df = pd.DataFrame(department_train_encoded,
                                          columns=ohe.get_feature_names_out(['Department']))
department_test_encoded_df = pd.DataFrame(department_test_encoded,
                                        columns=ohe.get_feature_names_out(['Department']))

# Display the first few rows of the new encoded DataFrames
print("Encoded Training Data for Department:")
print(department_train_encoded_df.head())

print("\nEncoded Testing Data for Department:")
print(department_test_encoded_df.head())


Encoded Training Data for Department:
   Department_Research & Development  Department_Sales
0                                1.0               0.0
1                                0.0               1.0
2                                0.0               1.0
3                                0.0               1.0
4                                0.0               1.0

Encoded Testing Data for Department:
   Department_Research & Development  Department_Sales
0                                0.0               1.0
1                                1.0               0.0
2                                0.0               0.0
3                                1.0               0.0
4                                1.0               0.0


In [84]:
department_train_encoded_df.shape

(1102, 2)

In [76]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the testing data
X_test_scaled = scaler.transform(X_test)


In [77]:
# Create a OneHotEncoder for the Attrition column
ohe = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
ohe.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
# Transform the training data
attrition_train_encoded = ohe.transform(y_train[['Attrition']])
# Transform the testing data
attrition_test_encoded = ohe.transform(y_test[['Attrition']])

# Create new variables to hold the encoded results
# Convert the encoded arrays to DataFrames for easier handling
attrition_train_encoded_df = pd.DataFrame(attrition_train_encoded,
                                          columns=ohe.get_feature_names_out(['Attrition']))
attrition_test_encoded_df = pd.DataFrame(attrition_test_encoded,
                                          columns=ohe.get_feature_names_out(['Attrition']))

# Display the first few rows of the new encoded DataFrames
print("Encoded Training Data for Attrition:")
print(attrition_train_encoded_df.head())

print("\nEncoded Testing Data for Attrition:")
print(attrition_test_encoded_df.head())



Encoded Training Data for Attrition:
   Attrition_Yes
0            0.0
1            0.0
2            0.0
3            0.0
4            0.0

Encoded Testing Data for Attrition:
   Attrition_Yes
0            0.0
1            0.0
2            1.0
3            0.0
4            0.0


In [85]:
attrition_train_encoded_df.shape

(1102, 1)

## Create, Compile, and Train the Model

In [78]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]
num_columns

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))

# Create the hidden layer
hidden_layer = layers.Dense(units=128, activation='relu')(input_layer)

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)

In [90]:
# Create a branch for Department
# with a hidden layer and an output layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(2, activation='softmax', name='department_output')(department_hidden)

In [91]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [92]:
# Create the model
model = models.Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 10)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared1 (Dense)           │ (None, 64)             │            704 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared2 (Dense)           │ (None, 128)            │          8,320 │ shared1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden (Dense) │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden (Dense)  │ (None, 32)             │          4,128 │ shared2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 2)              │             66 │ department_hidden[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 1)              │             33 │ attrition_hidden[0][0] │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,379 (67.89 KB)

 Trainable params: 17,379 (67.89 KB)

 Non-trainable params: 0 (0.00 B)

In [94]:
# Train the model
model.fit(X_train_scaled,
          [attrition_train_encoded, department_train_encoded],
          epochs=100,
          batch_size=32)


Epoch 1/100


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 2)

In [ ]:
# Evaluate the model with the testing data
scores = model.evaluate(X_test_scaled, [attrition_test_encoded, department_test_encoded])


In [ ]:
# Print the accuracy for both department and attrition
print(scores)

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.